In [1]:
#pour un spectacle voir les coproductions
#https://www.lesarchivesduspectacle.net/?IDX_Spectacle=90517#spectacle-partenaires-de-production
#Perchée dans les arbres 8 coproductions 

import urllib.request
import re
import pandas as pd
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup
#spectacleId="34525"
spectacleId="36956"
#spectacleId="90517"
# specify which URL/web page we are going to be scraping
url = "https://www.lesarchivesduspectacle.net/?IDX_Spectacle=" + spectacleId +"#spectacle-partenaires-de-production"
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")
#the div where the information is
#the div where the information is
table = soup.find('table', class_='f-spectacle__production-bis') 
# Down below we'll add our inmates to this list
inmates_list = []

# BeautifulSoup provides nice ways to access the data in the parsed
# page. Here, we'll use the select method and pass it a CSS style
# selector to grab all the rows in the table (the rows contain the 
# inmate names and ages).

for table_row in table.find_all('tr'):
    # Each tr (table row) has three td HTML elements (most people 
    # call these table cels) in it (first name, last name, and age)
    cells = table_row.findAll('td')
    # Our table has one exception -- a row without any cells.
    # Let's handle that special case here by making sure we
    # have more than zero cells before processing the cells
    if len(cells) > 0:
        organism = cells[1].text.strip()
        organism_element = cells[1].find('a',class_='c_Organisme b')
        organismId = organism_element['href'].replace("?IDX_Organisme=","")
        type_aide = cells[0].text.strip()
        if(type_aide != ''):
            type_aide_master = type_aide
        
        else:
            type_aide = type_aide_master
        inmate = {'performanceId': spectacleId,'type_aide': type_aide,'organism': organism,'organismId': organismId}
        inmates_list.append(inmate)

df = pd.DataFrame(inmates_list)
df


,performanceId,type_aide,organism,organismId
0,36956,Coproduction,Théâtre National de Chaillot (Paris),585
1,36956,Coproduction,HAU Theater Hebbel am Ufer (Berlin),206
2,36956,Coproduction,Grand Théâtre de la Ville de Luxembourg (Luxe...,383
3,36956,Coproduction,La Rose des Vents (Villeneuve-d'Ascq),141
4,36956,Coproduction,Centro Cultural Vila Flor (Guimarães),12754
5,36956,Diffusion,Frans Brood Productions (Gand),4312


In [7]:
#function return dataframe with type_aide and organism
import urllib.request
import re
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup
import pandas as pd


def import_df_type_production(spectacleId,saison):
    # specify which URL/web page we are going to be scraping
    url = "https://www.lesarchivesduspectacle.net/?IDX_Spectacle=" + spectacleId +"#spectacle-partenaires-de-production"
    try:
        page = urllib.request.urlopen(url)
        # parse the HTML from our URL into the BeautifulSoup parse tree format
        soup = BeautifulSoup(page, "lxml")
        table = soup.find('table', class_='f-spectacle__production-bis') 
        # Down below we'll add our inmates to this list
        inmates_list = []
        if table is not None:
            for table_row in table.find_all('tr'):
                if table_row is not None:
                    cells = table_row.findAll('td')
                    if len(cells) > 0:
                        organism = cells[1].text.strip()
                        organism_element = cells[1].find('a',class_='c_Organisme b')
                        organismId = organism_element['href'].replace("?IDX_Organisme=","")
                        type_aide = cells[0].text.strip()
                        if(type_aide != ''):
                            type_aide_master = type_aide 
                        else:
                            type_aide = type_aide_master
                        inmate = {'performanceId': spectacleId,'type_aide': type_aide,'organism': organism,'organismId': organismId,'saison':saison}
                        inmates_list.append(inmate)
                    else:
                        inmate = {'performanceId': spectacleId,'type_aide': '','organism': '','organismId': '','saison':saison}
                        inmates_list.append(inmate) 
            df = pd.DataFrame(inmates_list)
            return df
        else:
            inmate = {'performanceId': spectacleId,'type_aide': '','organism': '','organismId': '','saison':saison}
            inmates_list.append(inmate)
            df = pd.DataFrame(inmates_list)
            return df
    except urllib.error.URLError as e:
        print(e.reason + "for spectacleId "  + spectacleId + " saison " + saison)
    except socket.timeout as e: # <-------- this block here
        print("We timed out")

Unnamed: 0        int64
organism_id       int64
year              int64
performance      object
performanceId     int64
author           object
authordId         int64
label            object
period           object
occurency        object
place            object
dtype: object

In [8]:
#Pour les spectacles à l'Arsenal voir les types d'aide
# ORGANISM ID DE L'ETUDE
# 439 Arsenal
# 699 CCAM [Centre culturel André Malraux] - Scène Nationale de Vandoeuvre-lès-Nancy
# 383 Grand théâtre du Luxembourg
# 1914 Carreau forbach 
import time
from datetime import datetime
import pandas as pd
organismId="439"
csv_file_path = "all_10_years_organism_439.csv"
df_organism = pd.read_csv(csv_file_path)
df = pd.DataFrame(columns=['performanceId','type_aide','organism','organismId'])                        
print("start",datetime.now()) 
for row in df_organism.itertuples():
    if (row.performanceId!=0):
        #call the function to have the type_aide and orgnaism
        try:
            df=df.append(import_df_type_production(str(row.performanceId),row.year),sort=False,ignore_index=True)
        except Exception as err:
            continue
df.to_csv('all_10_years_organism_' + organismId + '_type_production.csv')

print("finish",datetime.now())

start 2020-07-26 11:28:28.638510
finish 2020-07-26 11:32:26.845880


In [16]:
import time
from datetime import datetime
import pandas as pd
csv_file_path = "all_10_years_organism_439_type_production.csv"
df_type_production = pd.read_csv(csv_file_path)

df_type_production = df_type_production[df_type_production['type_aide'] == 'Coproduction']
#where coproduction est Arsenal
df_type_production = df_type_production[df_type_production['organismId'] == 439.0]
df_type_production.groupby(["saison"]).performanceId.count()

saison
2010.0    1
2011.0    6
2012.0    4
2013.0    2
2014.0    4
2015.0    1
2016.0    1
2017.0    3
2018.0    3
Name: performanceId, dtype: int64

In [33]:
#2010    17
#2011    19
#2012    15
#2013    12
#2014    20
#2015    13
#2016    22
#2017    19
#2018    31
#2019    27
import time
from datetime import datetime
import pandas as pd
csv_file_path = "all_10_years_organism_439_type_production.csv"
df_type_production = pd.read_csv(csv_file_path)
df_type_production.type_aide.value_counts()


Coproduction     646
Soutien          247
Aide              67
Partenariat       23
Résidence         20
Collaboration      8
Participation      4
Diffusion          4
Coréalisation      1
Name: type_aide, dtype: int64

In [29]:
import time
from datetime import datetime
import pandas as pd
csv_file_path = "all_10_years_organism_439_type_production.csv"
df_type_production = pd.read_csv(csv_file_path)
#where organism is Arsenal
df_type_production = df_type_production[df_type_production['organismId'] == 439.0]
df_type_production.type_aide.value_counts()


Coproduction    25
Résidence        1
Partenariat      1
Name: type_aide, dtype: int64

In [34]:
df_type_production.nunique()

Unnamed: 0       1058
performanceId     192
type_aide           9
organism          535
organismId        533
saison             10
dtype: int64

In [36]:
import time
from datetime import datetime
import pandas as pd
csv_file_path = "all_10_years_organism_439.csv"
df_arsenal_10_years = pd.read_csv(csv_file_path)
df_arsenal_10_years.nunique()

Unnamed: 0       223
organism_id        1
year              10
performance      193
performanceId    193
author           141
authordId        142
label            199
period           158
occurency          9
place             11
dtype: int64